In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [27]:
train = pd.read_parquet('./train.parquet')
test = pd.read_parquet('./test.parquet')

# 변수탐색

0. id : 아이디

1. base_date : 날짜(int)
   - 281개의 특정 날짜만 사용
   - 2021.09.01~2022.07.31 간에 기록된 것이나 기록되지 않은 날도 존재

1. day_of_week : 요일(str)
   - 월~일 골고루 분포되어 있음

2. base_hour : 시간대(int)
   - 해가 떠있는 시간대에 표본이 많이 존재

3. lane_count : 차로 수(int)
   - 표본 수는 1차로(약 155만), 2차로(약 235만), 3차로(약 79만)

4. road_rating : 도로등급(int)
   - 표본 수는 103(약 215만), 106(약 95만), 107(약 158만)
   - 103은 일반국도, 106은 지방도, 107은 시도로 추정됨.

5. road_name : 도로명(str)
   - 약 56만개의 결측치 존재
   - 결측치 제외 60개의 도로명 존재 -> labeling 후 변수 사용에 대해 생각해봐야 됨.

6. multi_linked : 중용구간 여부(int)
   - 값이 1(약 0.2만), 0(약 469만)으로 이분

7. connect_code : 연결로 코드(int)
   - 연결로 코드가 0이 아닌 경우 중용여부는 무조건 0이다.(if connect_code!=0 then multi_linked==0)
   - 값이 103(약 1.2만), 0(약 468만)으로 이분

8. maximum_speed_limit : 최고속도 제한(float)
    - 30~80km/h로 나뉘어져 있음
    - 도로등급과 함께 보았을 때 고속국도는 없는 것으로 추정됨

9.  ~~vehicle_restricted : 통과차량제한(float)~~
    - 모든 값이 0임 -> 삭제해도 될 듯

10. weight_restricted : 통과제한하중(float)
    - 0(약 403만) 외 3개의 변수가 있음

11. ~~height_restricted : 통과제한높이(float)~~
    - 모든 값이 0임 -> 삭제해도 될듯

12. road_type : 도로유형(int)
    - 값이 0(약 373만), 3(약 96만)으로 이분

13. ~ 17.  start_node : 시작지점
    - name(str) : 지점명이 중복사용되는 경우가 다양하게 많음
    - lat(float), long(float) : 지점의 위도, 경도
    - restricted(str) : 회전제한 유무 // 없음(약 423만), 있음(약 46만) -> labeling해서 사용

14. ~ 21. end_node : 종료지점
    - name(str) : 지점명이 중복사용되는 경우가 다양하게 많음
    - lat(float), long(float) : 지점의 위도, 경도
    - restricted(str) : 회전제한 유무 // 없음(약 423만), 있음(약 46만) -> labeling해서 사용

    - 각 노드들을 하나의 객체로 봤을 때 start_node와 end_node가 중복사용이 많이 됨 -> 노드(이름,위도,경도,회전제한)를 하나의 객체로 구성해 사용하는 것이 바람직해보임.

15. target : 평균속도(km/h)(float)
    - 목적변수
    - train내에서만 존재



# 전처리 및 변수선택
- base_hour : 24개의 범주 -> 시간대별로 묶어(0~5, 6~10, 11~19, 20~23)
- road_name : 61개의 범주 + 각 범주값들간에 명확한 차이를 둘 수 없음 -> 삭제
- multi_linked : 데이터 극편향(이상치로 간주) + 상대적으로 높은 p-value + 후진 BIC에서 제거됨 -> 삭제
- connect_code : 데이터 극편향 + multi_linked와 관계있는 것으로 보임 -> 삭제
- weight_restricted : 제한없음에 편향 -> 제한유무로 변환
- start_node_name : 487개의 범주 + 각 범주값들 간 명확한 차이를 둘 수 없음 -> 삭제
- end_node_name : 487개의 범주 + 각 범주값들 간 명확한 차이를 둘 수 없음 -> 삭제
- start_turn_restricted : end_turn_restricted와 합쳐 4레벨의 범주형 변수로 변환